In [18]:
!wget -O file.zip "https://figshare.com/ndownloader/articles/4244171/versions/2"

--2024-03-02 05:40:36--  https://figshare.com/ndownloader/articles/4244171/versions/2
Resolving figshare.com (figshare.com)... 54.75.26.168, 79.125.17.12, 2a05:d018:1f4:d000:8474:96ea:8fe2:374a, ...
Connecting to figshare.com (figshare.com)|54.75.26.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 851639264 (812M) [application/zip]
Saving to: 'file.zip'

file.zip            100%[===================>] 812.19M  23.7MB/s    in 33s     

2024-03-02 05:41:10 (24.3 MB/s) - 'file.zip' saved [851639264/851639264]



In [19]:
!unzip -q file.zip -d dataset

replace dataset/H S1 EC.edf? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [20]:
!pip install mne

In [21]:
from glob import glob
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
#read all file
all_files_path=glob('dataset/*.edf')
print(len(all_files_path))

181


In [23]:
all_files_path[0]

'dataset/MDD S6 EO.edf'

In [24]:
healthy_file_path=[i for i in all_files_path if  'H' in i.split('/')[1] ]
patient_file_path=[i for i in all_files_path if  'MDD' in i.split('/')[1] ]
print(len(healthy_file_path),len(patient_file_path))

86 95


In [25]:
def read_data(file_path):
    datax=mne.io.read_raw_edf(file_path,preload=True)
    datax.set_eeg_reference()
    datax.filter(l_freq=1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(datax,duration=5,overlap=1)
    epochs=epochs.get_data()
    return epochs #trials,channel,length

In [26]:
data=read_data(healthy_file_path[0])

Extracting EDF parameters from /kaggle/working/dataset/H S28 TASK.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 155391  =      0.000 ...   606.996 secs...
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 845 samples (3.301 s)

Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using da

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [27]:
data.shape

(151, 22, 1280)

In [28]:
%%capture
control_epochs_array=[read_data(subject) for subject in healthy_file_path]
patients_epochs_array=[read_data(subject) for subject in patient_file_path]

In [29]:
control_epochs_labels=[len(i)*[0] for i in control_epochs_array]
patients_epochs_labels=[len(i)*[1] for i in patients_epochs_array]
print(len(control_epochs_labels),len(patients_epochs_labels))

86 95


In [30]:
data_list=control_epochs_array+patients_epochs_array
label_list=control_epochs_labels+patients_epochs_labels
print(len(data_list),len(label_list))

181 181


In [31]:
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]

In [32]:
# Find the maximum number of channels and length of epochs among all subjects
max_num_channels = max([epoch.shape[1] for epoch in data_list])
max_epochs_length = max([epoch.shape[2] for epoch in data_list])

# Pad or truncate epochs to the maximum number of channels and length
padded_data_list = []
for epoch in data_list:
    num_trials, num_channels, epoch_length = epoch.shape
    if num_channels < max_num_channels:
        # Pad the epoch along the channels dimension (axis=1)
        pad_width_channels = ((0, 0), (0, max_num_channels - num_channels), (0, 0))
        epoch = np.pad(epoch, pad_width_channels, mode='constant')
    elif num_channels > max_num_channels:
        # Truncate the epoch along the channels dimension (axis=1)
        epoch = epoch[:, :max_num_channels, :]

    if epoch_length < max_epochs_length:
        # Pad the epoch along the time dimension (axis=2)
        pad_width_time = ((0, 0), (0, 0), (0, max_epochs_length - epoch_length))
        epoch = np.pad(epoch, pad_width_time, mode='constant')
    elif epoch_length > max_epochs_length:
        # Truncate the epoch along the time dimension (axis=2)
        epoch = epoch[:, :, :max_epochs_length]

    padded_data_list.append(epoch)

In [33]:
data_array = np.vstack(padded_data_list)
label_array = np.hstack(label_list)
group_array = np.hstack(groups_list)
data_array=np.moveaxis(data_array,1,2)
print(data_array.shape,label_array.shape,group_array.shape)

(18387, 1280, 22) (18387,) (18387,)


### Model

In [37]:
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
def cnnmodel():
    clear_session()
    model=Sequential()
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,input_shape=(6250,22)))#1
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#3
    model.add(LeakyReLU())
    model.add(MaxPool1D(pool_size=2,strides=2))#4
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#6
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    model.add(LeakyReLU())
    model.add(AveragePooling1D(pool_size=2,strides=2))#8
    model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    model.add(LeakyReLU())
    model.add(GlobalAveragePooling1D())#10
    model.add(Dense(1,activation='sigmoid'))#11

    model.compile('adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model=cnnmodel()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 6248, 5)        │           335 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 6248, 5)        │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 6248, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 3124, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 3122, 5)        │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 3122, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1561, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1561, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1559, 5)        │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 1559, 5)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 779, 5)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 779, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 777, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 777, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d_1             │ (None, 388, 5)         │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 386, 5)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 386, 5)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 5)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 681 (2.66 KB)

 Trainable params: 671 (2.62 KB)

 Non-trainable params: 10 (40.00 B)

In [38]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()

In [43]:
accuracy=[]
for fold, (train_index, val_index) in enumerate(gkf.split(data_array, label_array, groups=group_array), 1):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)
    model=cnnmodel()
    model.fit(train_features,train_labels,epochs=50,batch_size=128,validation_data=(val_features,val_labels))
    acc = model.evaluate(val_features,val_labels)[1]
    accuracy.append(acc)
    model.save(f'model_fold_{fold}_acc_{int(acc*100)}.keras')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.6067 - loss: 0.6446 - val_accuracy: 0.9220 - val_loss: 0.3761
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.8621 - loss: 0.3717 - val_accuracy: 0.9394 - val_loss: 0.2362
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.8882 - loss: 0.2899 - val_accuracy: 0.9400 - val_loss: 0.2195
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.8845 - loss: 0.2869 - val_accuracy: 0.9408 - val_loss: 0.2274
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.8938 - loss: 0.2603 - val_accuracy: 0.8991 - val_loss: 0.2935
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9073 - loss: 0.2397 - val_accuracy: 0.8047 - val_loss: 0.3830
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9100 - loss: 0.2179 - val_accuracy: 0.5589 - val_loss: 0.5962
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9271 - loss: 0.1859 - val_a

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.7033 - loss: 0.6530 - val_accuracy: 0.8466 - val_loss: 0.3838
Epoch 2/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.8728 - loss: 0.3374 - val_accuracy: 0.8737 - val_loss: 0.3199
Epoch 3/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9017 - loss: 0.2685 - val_accuracy: 0.8707 - val_loss: 0.3185
Epoch 4/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9112 - loss: 0.2308 - val_accuracy: 0.6490 - val_loss: 0.6248
Epoch 5/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9233 - loss: 0.1965 - val_accuracy: 0.5416 - val_loss: 0.8379
Epoch 6/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9269 - loss: 0.1865 - val_accuracy: 0.4828 - val_loss: 1.7311
Epoch 7/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9409 - loss: 0.1544 - val_accuracy: 0.5041 - val_loss: 1.3313
Epoch 8/50
116/116 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9469 - loss: 0.1404 - val_ac

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.6877 - loss: 0.6534 - val_accuracy: 0.8270 - val_loss: 0.4044
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 9s 62ms/step - accuracy: 0.8663 - loss: 0.3311 - val_accuracy: 0.8922 - val_loss: 0.3042
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.8911 - loss: 0.2856 - val_accuracy: 0.8946 - val_loss: 0.2929
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.8972 - loss: 0.2611 - val_accuracy: 0.7368 - val_loss: 0.4614
Epoch 5/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9038 - loss: 0.2288 - val_accuracy: 0.5019 - val_loss: 0.8357
Epoch 6/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9184 - loss: 0.2018 - val_accuracy: 0.4651 - val_loss: 1.4117
Epoch 7/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9214 - loss: 0.1874 - val_accuracy: 0.4592 - val_loss: 1.5785
Epoch 8/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9313 - loss: 0.1598 - val_

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.5529 - loss: 0.6430 - val_accuracy: 0.9093 - val_loss: 0.3742
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.8577 - loss: 0.3938 - val_accuracy: 0.9210 - val_loss: 0.2497
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.8823 - loss: 0.3226 - val_accuracy: 0.9321 - val_loss: 0.2392
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.8861 - loss: 0.3083 - val_accuracy: 0.9331 - val_loss: 0.2324
Epoch 5/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.8932 - loss: 0.2867 - val_accuracy: 0.9277 - val_loss: 0.2485
Epoch 6/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9018 - loss: 0.2653 - val_accuracy: 0.9131 - val_loss: 0.2668
Epoch 7/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.9044 - loss: 0.2530 - val_accuracy: 0.8825 - val_loss: 0.3065
Epoch 8/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9073 - loss: 0.2500 - val_

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.7483 - loss: 0.6001 - val_accuracy: 0.7809 - val_loss: 0.4343
Epoch 2/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9032 - loss: 0.2668 - val_accuracy: 0.8558 - val_loss: 0.4125
Epoch 3/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 0.9197 - loss: 0.2076 - val_accuracy: 0.8425 - val_loss: 0.5792
Epoch 4/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.9410 - loss: 0.1537 - val_accuracy: 0.7836 - val_loss: 0.7913
Epoch 5/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9493 - loss: 0.1316 - val_accuracy: 0.8392 - val_loss: 0.8128
Epoch 6/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.9581 - loss: 0.1146 - val_accuracy: 0.8523 - val_loss: 0.8114
Epoch 7/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.9602 - loss: 0.1082 - val_accuracy: 0.8656 - val_loss: 0.7988
Epoch 8/50
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9659 - loss: 0.0945 - val

In [40]:
np.mean(accuracy)

0.9124922752380371

In [41]:
accuracy

[0.9599018096923828,
 0.9305631518363953,
 0.9045946002006531,
 0.936112642288208,
 0.8312891721725464]